# Reconciliation #08: Monte Carlo Convergence

## Overview
- **What this notebook tests:** Monte Carlo estimates (ruin probability, mean final wealth, growth rates) converge to stable values as sample size increases, and convergence diagnostics (MCSE, ESS) correctly indicate when enough simulations have been run.
- **Prerequisites:** `pip install -e .` from the project root.
- **Estimated runtime:** < 60 seconds
- **Audience:** Developers, actuaries, and risk managers verifying Monte Carlo convergence behaviour.

## Checks Performed
1. **MCSE Monotonicity:** Monte Carlo Standard Error decreases monotonically as sample size increases.
2. **MCSE Scaling:** MCSE decreases approximately as 1/sqrt(n).
3. **Estimate Stability:** Final estimates at n=1000 and n=2000 agree within reasonable tolerance.
4. **ESS Growth:** Effective Sample Size grows with sample size.
5. **Coefficient of Variation Decrease:** CV of running estimates decreases with more samples.

## Approach
We run individual `Simulation` instances at increasing sample sizes (50, 100, 200, 500, 1000, 2000) with different seeds, collecting key metrics (ruin probability, mean final wealth, simple growth measure) from each. At each sample size we compute convergence diagnostics using `ConvergenceDiagnostics` and verify that the estimates stabilize.

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/reconciliation'

    if os.path.exists(os.path.join(NOTEBOOK_DIR, '_reconciliation_helpers.py')):
        print(f"Found helper module in {NOTEBOOK_DIR}")
        os.chdir(NOTEBOOK_DIR)
        if NOTEBOOK_DIR not in sys.path:
            sys.path.append(NOTEBOOK_DIR)
    else:
        print(f"WARNING: _reconciliation_helpers.py not found in {NOTEBOOK_DIR}")
        print("Please update NOTEBOOK_DIR in this cell to the correct folder path.")

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
"""Setup: imports and notebook header."""
import sys
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Reconciliation helper utilities
sys.path.insert(0, os.path.dirname(os.path.abspath(".")))
from _reconciliation_helpers import (
    ReconciliationChecker, final_summary, section_header,
    notebook_header, timed_cell, fmt_dollar, display_df,
    create_standard_manufacturer, create_standard_loss_generator,
)

# Core framework imports
from ergodic_insurance.simulation import Simulation, SimulationResults
from ergodic_insurance.convergence import ConvergenceDiagnostics
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

# Display the standard notebook header
notebook_header(
    number=8,
    title="Monte Carlo Convergence",
    description=(
        "Verifies that Monte Carlo estimates (ruin probability, growth rates, mean final wealth) "
        "converge to stable values as sample size increases, and that convergence diagnostics "
        "(MCSE, ESS) correctly indicate when enough simulations have been run."
    ),
)

# Suppress verbose logging
import logging
logging.getLogger("ergodic_insurance").setLevel(logging.WARNING)
warnings.filterwarnings("ignore")

plt.style.use("seaborn-v0_8-darkgrid")

# Fixed seed for reproducibility
BASE_SEED = 42
np.random.seed(BASE_SEED)

print("Setup complete.")

## Section 1: Configure Base Scenario

We create a standard manufacturer and loss generator configuration that will be
used across all sample sizes. Each simulation uses a 10-year time horizon with
different seeds to produce independent paths.

In [ ]:
"""Configure the base scenario for convergence experiments."""
section_header("1. Configure Base Scenario")

# Simulation parameters
TIME_HORIZON = 10  # years per simulation path
SAMPLE_SIZES = [50, 100, 200, 500, 1000, 2000]
MAX_SAMPLES = max(SAMPLE_SIZES)  # 2000

# Manufacturer parameters (standard reconciliation defaults)
INITIAL_ASSETS = 10_000_000

print(f"Time horizon:    {TIME_HORIZON} years per path")
print(f"Sample sizes:    {SAMPLE_SIZES}")
print(f"Max samples:     {MAX_SAMPLES}")
print(f"Initial assets:  {fmt_dollar(INITIAL_ASSETS)}")
print(f"Base seed:       {BASE_SEED}")

## Section 2: Run Convergence Experiment

We run the maximum number of simulations (2000) in a single pass, collecting
key metrics from each path. We then compute statistics at each sample size
by using the first N results.

In [ ]:
"""Run 2000 independent simulation paths and collect metrics."""
section_header("2. Run Convergence Experiment")

with timed_cell("Monte Carlo simulation (2000 paths)"):
    # Storage for per-path metrics
    final_wealth = np.zeros(MAX_SAMPLES)
    survived = np.zeros(MAX_SAMPLES, dtype=bool)
    growth_rates = np.zeros(MAX_SAMPLES)

    for i in range(MAX_SAMPLES):
        # Create fresh manufacturer and loss generator with unique seed
        seed_i = BASE_SEED + i
        mfr = create_standard_manufacturer(initial_assets=INITIAL_ASSETS)
        loss_gen = create_standard_loss_generator(seed=seed_i)

        # Run simulation
        sim = Simulation(
            manufacturer=mfr,
            loss_generator=loss_gen,
            time_horizon=TIME_HORIZON,
            seed=seed_i,
            copy=False,  # We already created a fresh manufacturer
        )
        result = sim.run()

        # Collect metrics
        fw = float(result.equity[-1])
        final_wealth[i] = fw
        survived[i] = result.insolvency_year is None

        # Compute simple annualized growth rate
        if fw > 0 and result.insolvency_year is None:
            growth_rates[i] = (fw / INITIAL_ASSETS) ** (1.0 / TIME_HORIZON) - 1.0
        else:
            growth_rates[i] = -1.0  # total loss

print(f"\nCompleted {MAX_SAMPLES} simulation paths.")
print(f"Survival rate:     {np.mean(survived):.1%}")
print(f"Mean final wealth: {fmt_dollar(np.mean(final_wealth))}")
print(f"Median growth:     {np.median(growth_rates[survived]):.2%}")

In [ ]:
"""Compute running estimates and diagnostics at each sample size."""

# Initialize ConvergenceDiagnostics with relaxed thresholds for small samples
diag = ConvergenceDiagnostics(
    r_hat_threshold=1.1,
    min_ess=100,  # Relaxed for our small sample sizes
    relative_mcse_threshold=0.10,
)

# Storage for metrics at each sample size
results_table = []

for n in SAMPLE_SIZES:
    # Use the first n samples
    fw_n = final_wealth[:n]
    surv_n = survived[:n]
    gr_n = growth_rates[:n]

    # --- Key estimates ---
    ruin_prob = 1.0 - np.mean(surv_n)
    mean_fw = np.mean(fw_n)
    mean_gr = np.mean(gr_n[surv_n]) if np.any(surv_n) else np.nan

    # --- MCSE for each metric ---
    # For final wealth (continuous): use ConvergenceDiagnostics
    mcse_fw = diag.calculate_mcse(fw_n)
    ess_fw = diag.calculate_ess(fw_n)

    # For growth rate (continuous, survivors only)
    gr_survivors = gr_n[surv_n]
    if len(gr_survivors) > 4:
        mcse_gr = diag.calculate_mcse(gr_survivors)
        ess_gr = diag.calculate_ess(gr_survivors)
    else:
        mcse_gr = np.nan
        ess_gr = np.nan

    # For ruin probability (binary): MCSE = sqrt(p*(1-p)/n)
    mcse_ruin = np.sqrt(ruin_prob * (1 - ruin_prob) / n) if 0 < ruin_prob < 1 else 0.0

    # Coefficient of variation of the running mean (stability measure)
    # Use rolling sub-samples to estimate CV
    if n >= 50:
        n_blocks = min(10, n // 10)
        block_size = n // n_blocks
        block_means = [np.mean(fw_n[j*block_size:(j+1)*block_size]) for j in range(n_blocks)]
        cv_fw = np.std(block_means) / np.mean(block_means) if np.mean(block_means) > 0 else np.inf
    else:
        cv_fw = np.nan

    results_table.append({
        "n": n,
        "ruin_prob": ruin_prob,
        "mean_final_wealth": mean_fw,
        "mean_growth_rate": mean_gr,
        "mcse_ruin": mcse_ruin,
        "mcse_fw": mcse_fw,
        "mcse_gr": mcse_gr,
        "ess_fw": ess_fw,
        "ess_gr": ess_gr,
        "cv_fw": cv_fw,
    })

df_results = pd.DataFrame(results_table)

# Display summary table
df_display = df_results.copy()
df_display["ruin_prob"] = df_display["ruin_prob"].map(lambda x: f"{x:.3%}")
df_display["mean_final_wealth"] = df_display["mean_final_wealth"].map(lambda x: f"${x:,.0f}")
df_display["mean_growth_rate"] = df_display["mean_growth_rate"].map(
    lambda x: f"{x:.3%}" if not np.isnan(x) else "N/A"
)
df_display["mcse_ruin"] = df_display["mcse_ruin"].map(lambda x: f"{x:.4f}")
df_display["mcse_fw"] = df_display["mcse_fw"].map(lambda x: f"${x:,.0f}")
df_display["mcse_gr"] = df_display["mcse_gr"].map(
    lambda x: f"{x:.5f}" if not np.isnan(x) else "N/A"
)
df_display["ess_fw"] = df_display["ess_fw"].map(lambda x: f"{x:.0f}")
df_display["ess_gr"] = df_display["ess_gr"].map(
    lambda x: f"{x:.0f}" if not np.isnan(x) else "N/A"
)
df_display["cv_fw"] = df_display["cv_fw"].map(
    lambda x: f"{x:.4f}" if not np.isnan(x) else "N/A"
)

display_df(df_display, title="Convergence Metrics by Sample Size")

## Section 3: MCSE Analysis

Verify that Monte Carlo Standard Error (MCSE) decreases monotonically with sample
size and approximately follows the theoretical 1/sqrt(n) scaling. A monotonically
decreasing MCSE is the fundamental indicator that additional samples improve precision.

In [ ]:
"""MCSE monotonicity and scaling checks."""
section_header("3. MCSE Analysis")

checker_mcse = ReconciliationChecker(section="MCSE Analysis")

with timed_cell("MCSE checks"):
    mcse_fw_values = df_results["mcse_fw"].values
    mcse_ruin_values = df_results["mcse_ruin"].values
    sample_sizes_arr = df_results["n"].values

    # Check 1: MCSE for final wealth decreases monotonically
    fw_monotonic = all(
        mcse_fw_values[i] >= mcse_fw_values[i + 1]
        for i in range(len(mcse_fw_values) - 1)
    )
    checker_mcse.check(
        fw_monotonic,
        message="MCSE(final wealth) decreases monotonically with sample size",
        detail=f"Values: {[f'${v:,.0f}' for v in mcse_fw_values]}",
    )

    # Check 2: MCSE for ruin probability decreases monotonically
    # Filter to sample sizes where ruin occurs (non-zero MCSE)
    nonzero_ruin = mcse_ruin_values[mcse_ruin_values > 0]
    if len(nonzero_ruin) > 1:
        ruin_monotonic = all(
            nonzero_ruin[i] >= nonzero_ruin[i + 1]
            for i in range(len(nonzero_ruin) - 1)
        )
    else:
        ruin_monotonic = True  # Trivially true if zero/one non-zero
    checker_mcse.check(
        ruin_monotonic,
        message="MCSE(ruin prob) decreases monotonically with sample size",
        detail=f"Values: {[f'{v:.4f}' for v in mcse_ruin_values]}",
    )

    # Check 3: MCSE scales approximately as 1/sqrt(n)
    # Compare MCSE ratio between n=200 and n=2000 to theoretical sqrt(200/2000)
    idx_200 = list(SAMPLE_SIZES).index(200)
    idx_2000 = list(SAMPLE_SIZES).index(2000)
    actual_ratio = mcse_fw_values[idx_2000] / mcse_fw_values[idx_200]
    theoretical_ratio = np.sqrt(200.0 / 2000.0)  # = sqrt(0.1) ~ 0.316

    # Allow 50% tolerance on the ratio (Monte Carlo has sampling variability)
    ratio_close = abs(actual_ratio - theoretical_ratio) / theoretical_ratio < 0.50
    checker_mcse.check(
        ratio_close,
        message="MCSE scales approximately as 1/sqrt(n)",
        detail=(
            f"Actual ratio MCSE(2000)/MCSE(200) = {actual_ratio:.3f}, "
            f"theoretical = {theoretical_ratio:.3f}, "
            f"relative error = {abs(actual_ratio - theoretical_ratio) / theoretical_ratio:.1%}"
        ),
    )

    # Check 4: MCSE at n=2000 is at least 3x smaller than at n=50
    idx_50 = list(SAMPLE_SIZES).index(50)
    reduction_factor = mcse_fw_values[idx_50] / mcse_fw_values[idx_2000]
    checker_mcse.check(
        reduction_factor >= 3.0,
        message="MCSE at n=2000 is at least 3x smaller than at n=50",
        detail=f"Reduction factor: {reduction_factor:.1f}x",
    )

checker_mcse.display_results()

In [ ]:
"""Convergence plots: running estimates with error bands and MCSE decay."""

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# --- Plot 1: Running mean of final wealth ---
ax = axes[0, 0]
running_mean_fw = np.cumsum(final_wealth) / np.arange(1, MAX_SAMPLES + 1)
running_std_fw = np.array([final_wealth[:i+1].std(ddof=1) if i > 0 else 0
                           for i in range(MAX_SAMPLES)])
se_fw = running_std_fw / np.sqrt(np.arange(1, MAX_SAMPLES + 1))
n_range = np.arange(1, MAX_SAMPLES + 1)
ax.plot(n_range, running_mean_fw, lw=1.2, color="steelblue", label="Running mean")
ax.fill_between(n_range, running_mean_fw - 2 * se_fw, running_mean_fw + 2 * se_fw,
                alpha=0.2, color="steelblue", label="95% CI")
ax.axhline(running_mean_fw[-1], ls="--", color="red", alpha=0.5,
           label=f"Final: {fmt_dollar(running_mean_fw[-1])}")
ax.set_xlabel("Number of Simulations")
ax.set_ylabel("Mean Final Wealth ($)")
ax.set_title("Running Mean: Final Wealth")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# --- Plot 2: Running ruin probability ---
ax = axes[0, 1]
running_ruin = np.cumsum(~survived) / np.arange(1, MAX_SAMPLES + 1)
ruin_se = np.sqrt(running_ruin * (1 - running_ruin) / np.arange(1, MAX_SAMPLES + 1))
ax.plot(n_range, running_ruin * 100, lw=1.2, color="darkorange", label="Running estimate")
ax.fill_between(n_range, (running_ruin - 2 * ruin_se) * 100,
                (running_ruin + 2 * ruin_se) * 100,
                alpha=0.2, color="darkorange", label="95% CI")
ax.axhline(running_ruin[-1] * 100, ls="--", color="red", alpha=0.5,
           label=f"Final: {running_ruin[-1]:.2%}")
ax.set_xlabel("Number of Simulations")
ax.set_ylabel("Ruin Probability (%)")
ax.set_title("Running Estimate: Ruin Probability")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# --- Plot 3: MCSE decay for final wealth ---
ax = axes[1, 0]
ax.plot(SAMPLE_SIZES, mcse_fw_values, "o-", color="green", lw=2, label="Observed MCSE")
# Theoretical 1/sqrt(n) curve (scaled to match n=50 value)
theoretical_mcse = mcse_fw_values[0] * np.sqrt(SAMPLE_SIZES[0]) / np.sqrt(SAMPLE_SIZES)
ax.plot(SAMPLE_SIZES, theoretical_mcse, "--", color="gray",
        lw=1.5, label=r"Theoretical $1/\sqrt{n}$")
ax.set_xlabel("Sample Size")
ax.set_ylabel("MCSE ($)")
ax.set_title("MCSE Decay: Final Wealth")
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# --- Plot 4: Running mean of growth rate (survivors) ---
ax = axes[1, 1]
# Compute running mean over survivors only
surv_gr = growth_rates[survived]
if len(surv_gr) > 0:
    running_mean_gr = np.cumsum(surv_gr) / np.arange(1, len(surv_gr) + 1)
    running_std_gr = np.array([surv_gr[:i+1].std(ddof=1) if i > 0 else 0
                               for i in range(len(surv_gr))])
    se_gr = running_std_gr / np.sqrt(np.arange(1, len(surv_gr) + 1))
    n_surv = np.arange(1, len(surv_gr) + 1)
    ax.plot(n_surv, running_mean_gr * 100, lw=1.2, color="purple", label="Running mean")
    ax.fill_between(n_surv, (running_mean_gr - 2 * se_gr) * 100,
                    (running_mean_gr + 2 * se_gr) * 100,
                    alpha=0.2, color="purple", label="95% CI")
    ax.axhline(running_mean_gr[-1] * 100, ls="--", color="red", alpha=0.5,
               label=f"Final: {running_mean_gr[-1]:.2%}")
ax.set_xlabel("Number of Surviving Paths")
ax.set_ylabel("Mean Growth Rate (%)")
ax.set_title("Running Mean: Growth Rate (Survivors)")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

plt.suptitle("Monte Carlo Convergence Diagnostics", fontweight="bold", fontsize=14)
plt.tight_layout()
plt.show()

## Section 4: ESS Analysis

Verify that Effective Sample Size (ESS) grows with the nominal sample size.
ESS accounts for autocorrelation in the sample chain. For independent Monte Carlo
draws, ESS should be close to the nominal sample size.

In [ ]:
"""ESS checks: verify ESS grows with sample size."""
section_header("4. ESS Analysis")

checker_ess = ReconciliationChecker(section="ESS Analysis")

with timed_cell("ESS checks"):
    ess_fw_values = df_results["ess_fw"].values

    # Check 1: ESS increases with sample size (monotonically)
    ess_increasing = all(
        ess_fw_values[i] <= ess_fw_values[i + 1]
        for i in range(len(ess_fw_values) - 1)
    )
    checker_ess.check(
        ess_increasing,
        message="ESS(final wealth) increases monotonically with sample size",
        detail=f"Values: {[f'{v:.0f}' for v in ess_fw_values]}",
    )

    # Check 2: ESS at n=2000 is at least 500
    ess_at_2000 = ess_fw_values[-1]
    checker_ess.check(
        ess_at_2000 >= 500,
        message="ESS at n=2000 is at least 500 (reasonable efficiency)",
        detail=f"ESS = {ess_at_2000:.0f}",
    )

    # Check 3: ESS efficiency (ESS/n) is at least 25%
    # For independent draws, efficiency should be near 100%, but with
    # heavy-tailed distributions the initial-positive-sequence estimator
    # may report lower values.
    efficiency = ess_at_2000 / 2000.0
    checker_ess.check(
        efficiency >= 0.25,
        message="ESS efficiency at n=2000 is at least 25%",
        detail=f"Efficiency = {efficiency:.1%} (ESS={ess_at_2000:.0f}, n=2000)",
    )

    # Check 4: ESS for growth rate (survivors) also grows
    ess_gr_values = df_results["ess_gr"].dropna().values
    if len(ess_gr_values) > 1:
        ess_gr_increasing = all(
            ess_gr_values[i] <= ess_gr_values[i + 1]
            for i in range(len(ess_gr_values) - 1)
        )
        checker_ess.check(
            ess_gr_increasing,
            message="ESS(growth rate) increases with sample size",
            detail=f"Values: {[f'{v:.0f}' for v in ess_gr_values]}",
        )
    else:
        checker_ess.check(
            True,
            message="ESS(growth rate) increases with sample size (too few data points)",
            detail="Insufficient survivor data at multiple sample sizes",
        )

# Display ESS summary
ess_df = df_results[["n", "ess_fw", "ess_gr"]].copy()
ess_df["ess_efficiency"] = ess_df["ess_fw"] / ess_df["n"]
ess_df["ess_fw"] = ess_df["ess_fw"].map(lambda x: f"{x:.0f}")
ess_df["ess_gr"] = ess_df["ess_gr"].map(lambda x: f"{x:.0f}" if not np.isnan(x) else "N/A")
ess_df["ess_efficiency"] = ess_df["ess_efficiency"].map(lambda x: f"{x:.1%}")
display_df(ess_df, title="Effective Sample Size Summary")

checker_ess.display_results()

## Section 5: Stability Check

Verify that estimates stabilize as sample size increases. Specifically:
- Final estimates at n=1000 and n=2000 agree within reasonable tolerance.
- Coefficient of variation of block means decreases with sample size.

In [ ]:
"""Stability checks: estimates at n=1000 and n=2000 should agree."""
section_header("5. Stability Check")

checker_stability = ReconciliationChecker(section="Estimate Stability")

with timed_cell("Stability checks"):
    idx_1000 = list(SAMPLE_SIZES).index(1000)
    idx_2000 = list(SAMPLE_SIZES).index(2000)

    # --- Final wealth stability ---
    fw_1000 = df_results.iloc[idx_1000]["mean_final_wealth"]
    fw_2000 = df_results.iloc[idx_2000]["mean_final_wealth"]
    # Tolerance: 10% relative (reasonable for 1000 vs 2000 samples)
    rel_diff_fw = abs(fw_2000 - fw_1000) / abs(fw_2000) if fw_2000 != 0 else np.inf
    checker_stability.check(
        rel_diff_fw < 0.10,
        message="Mean final wealth at n=1000 and n=2000 agree within 10%",
        detail=f"n=1000: {fmt_dollar(fw_1000)}, n=2000: {fmt_dollar(fw_2000)}, relative diff: {rel_diff_fw:.2%}",
    )

    # --- Ruin probability stability ---
    ruin_1000 = df_results.iloc[idx_1000]["ruin_prob"]
    ruin_2000 = df_results.iloc[idx_2000]["ruin_prob"]
    # Absolute tolerance for ruin (which can be near zero)
    abs_diff_ruin = abs(ruin_2000 - ruin_1000)
    checker_stability.check(
        abs_diff_ruin < 0.05,
        message="Ruin probability at n=1000 and n=2000 agree within 5pp",
        detail=f"n=1000: {ruin_1000:.3%}, n=2000: {ruin_2000:.3%}, abs diff: {abs_diff_ruin:.3%}",
    )

    # --- Growth rate stability ---
    gr_1000 = df_results.iloc[idx_1000]["mean_growth_rate"]
    gr_2000 = df_results.iloc[idx_2000]["mean_growth_rate"]
    if not np.isnan(gr_1000) and not np.isnan(gr_2000):
        abs_diff_gr = abs(gr_2000 - gr_1000)
        checker_stability.check(
            abs_diff_gr < 0.02,
            message="Mean growth rate at n=1000 and n=2000 agree within 2pp",
            detail=f"n=1000: {gr_1000:.3%}, n=2000: {gr_2000:.3%}, abs diff: {abs_diff_gr:.3%}",
        )
    else:
        checker_stability.check(
            True,
            message="Mean growth rate stability (insufficient survivor data)",
            detail="Cannot compare: one or both sample sizes have NaN growth rate",
        )

    # --- Coefficient of variation decreases ---
    cv_values = df_results["cv_fw"].dropna().values
    if len(cv_values) >= 2:
        cv_first = cv_values[0]
        cv_last = cv_values[-1]
        checker_stability.check(
            cv_last < cv_first,
            message="CV of block means decreases from smallest to largest sample size",
            detail=f"CV(n={SAMPLE_SIZES[0]}) = {cv_first:.4f}, CV(n={SAMPLE_SIZES[-1]}) = {cv_last:.4f}",
        )
    else:
        checker_stability.check(
            True,
            message="CV decrease check (insufficient data points)",
            detail="Need at least 2 non-NaN CV values",
        )

checker_stability.display_results()

In [ ]:
"""Run full ConvergenceDiagnostics.check_convergence on the largest sample."""

print("Full convergence diagnostics on n=2000 sample:")
print("=" * 55)

# Create two independent chains from the full sample for R-hat check
chain1 = final_wealth[:1000]
chain2 = final_wealth[1000:2000]
chains = np.array([chain1, chain2])

# Use check_convergence which expects (n_chains, n_iterations) or (n_chains, n_iterations, n_metrics)
# We'll pass shape (2, 1000, 2) for two metrics: final_wealth and growth_rate
gr_chain1 = growth_rates[:1000]
gr_chain2 = growth_rates[1000:2000]

multi_chains = np.stack(
    [np.column_stack([chain1, gr_chain1]),
     np.column_stack([chain2, gr_chain2])],
    axis=0,
)  # shape: (2, 1000, 2)

conv_results = diag.check_convergence(
    multi_chains,
    metric_names=["final_wealth", "growth_rate"],
)

for name, stats in conv_results.items():
    print(f"\n{name}:")
    print(f"  R-hat:           {stats.r_hat:.4f}")
    print(f"  ESS:             {stats.ess:.0f}")
    print(f"  MCSE:            {stats.mcse:.6f}")
    print(f"  Autocorrelation: {stats.autocorrelation:.4f}")
    print(f"  Converged:       {stats.converged}")

## Section 6: Combined Summary

A consolidated view of all convergence checks, providing a single dashboard
to confirm that Monte Carlo estimates converge properly.

In [ ]:
"""Combined summary of all convergence checks."""
section_header("6. Combined Summary")

sections = [
    ("MCSE Analysis", checker_mcse),
    ("ESS Analysis", checker_ess),
    ("Estimate Stability", checker_stability),
]

summary_rows = []
for name, checker in sections:
    passed, failed = checker.summary_counts
    total = passed + failed
    summary_rows.append({
        "Check Section": name,
        "Passed": passed,
        "Failed": failed,
        "Total": total,
        "Status": "ALL PASS" if failed == 0 else f"{failed} FAILED",
    })

df_summary = pd.DataFrame(summary_rows)
display_df(df_summary, title="Convergence Check Summary")

total_passed = sum(r["Passed"] for r in summary_rows)
total_failed = sum(r["Failed"] for r in summary_rows)
total_checks = total_passed + total_failed
print(f"\nTotal: {total_passed}/{total_checks} checks passed")
if total_failed == 0:
    print("All Monte Carlo convergence checks PASSED.")
else:
    print(f"WARNING: {total_failed} check(s) FAILED.")

## Final Result

The cell below produces the final PASS/FAIL banner. If any check failed,
it raises an `AssertionError` so that `nbconvert` or CI will catch the failure.

In [ ]:
"""Final PASS/FAIL summary -- raises AssertionError on failure."""
final_summary(checker_mcse, checker_ess, checker_stability)